In [2]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import json
import time
import pickle
import os
import skimage as sk
from skimage import io
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim


with open('secret.json') as json_file:
    env_variables = json.load(json_file)
    os.environ['EMAIL_ADD'] = env_variables['EMAIL_ADD']
    os.environ['EMAIL_PASS'] = env_variables['EMAIL_PASS']
    os.environ['AZURE'] = env_variables['AZURE']

In [3]:
azure_key = os.environ.get('AZURE')
blob_service_client = BlobServiceClient.from_connection_string(azure_key)

In [4]:
containers = blob_service_client.list_containers() 
cs = {}
for c in containers:
    cs[c.name] = c.last_modified
container_name = sorted(cs, key=cs.get, reverse=True)[0]

In [5]:
container_name

'containerb24e111d-6a02-46db-a660-676585ad6537'

In [11]:
image_history = []
SSIM_THRESH = 0.5

while True:
    time.sleep(5)
    # new container client
    container_client = blob_service_client.get_container_client(container_name)
    container_client.list_blobs()
    blobs = []
    for i in container_client.list_blobs():
        blobs.append(i)
    print("Num blobs: {}".format(len(blobs)))
    
    for blob in container_client.list_blobs():
        name = blob['name']
        if name not in image_history:
            blob_client = blob_service_client.get_blob_client(container=container_name, blob=name)
            # save current image
            with open('real_time/{}.jpg'.format(int(time.time())), 'wb') as f:
                f.write(blob_client.download_blob().readall())
            time.sleep(1)
            if len(os.listdir('real_time'))!=0:
                last_img = sorted(os.listdir('real_time'))[-2]
                if last_img != '.DS_Store':
                    last_img = io.imread('real_time/{}'.format(last_img))
                    current_img = sorted(os.listdir('real_time'))[-1]
                    current_img = io.imread('real_time/{}'.format(current_img))
                    sim_idx = ssim(last_img, current_img, multichannel=True)
                    print(sim_idx)
                    if sim_idx <= 0.7:
                        print("New event found!!!!")
            image_history.append(name)
    

Num blobs: 0
Num blobs: 0
Num blobs: 1
0.558938954092395
New event found!!!!
Num blobs: 1
Num blobs: 2
1.0
Num blobs: 3
0.6443994871672962
New event found!!!!
Num blobs: 4
0.7386093829067978
Num blobs: 5
0.724245107258496
Num blobs: 5


KeyboardInterrupt: 

In [12]:
current_img = io.imread('real_time/1579399382.jpg')
last_img = io.imread('real_time/1579399390.jpg')

In [14]:
import numpy as np
from scipy import signal

In [ ]:
corr = signal.correlate2d(current_img[:,:,0], last_img[:,:,0])

In [ ]:
plt.imshow(corr, cmap='gray')